In [3]:
import os
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser

In [16]:
def get_statics(datas,channelId,playlistName):
    category_id=[]
    views=[]
    likes=[]
    favorites = []
    comments=[]
    title=[]
    date=[]
    tag=[]

    for i in range(len(datas)):
        request=youtube.videos().list(
        part='snippet,statistics',
        id=datas['video_id'][i],
        maxResults=100)
        
        response=request.execute()
        print(response['items'][0]['snippet'].keys())
        
        if response['items']==[]:
            title.append('-')
            category_id.append('-')
            views.append('-')
            likes.append('-')
            favorites.append('-')
            comments.append('-')
            date.append('-')
            tag.aapend('-')
            
        else :
            title.append(response['items'][0]['snippet']['title'])
            category_id.append(response['items'][0]['snippet']['categoryId'])
            views.append(response['items'][0]['statistics']['viewCount'])
            likes.append(response['items'][0]['statistics']['likeCount'])
            favorites.append(response['items'][0]['statistics']['favoriteCount'])
            comments.append(response['items'][0]['statistics']['commentCount'])
            date.append(response['items'][0]['snippet']['publishedAt'])
            tag.append(response['items'][0]['snippet']['tags'])
        
    df=pd.DataFrame([title,category_id,views,likes,comments,date]).T
    df.columns=['title','category_id','views','likes','favorites','comments','date','tags']
    today = date.today()

    currentDay = today.strftime("%Y%m%d")

    df.to_csv(f'./DATA/{channleId}_{playlistName}_{currentDay}.csv', sep=',', na_rep='NaN')
        

In [17]:


path = ".\DATA\ids"
file_lst = os.listdir(path)
f = open("key.txt",'r')
DEVELOPER_KEY= f.readline()
f.close()

YOUTUBE_API_SERVICE_NAME='youtube'
YOUTUBE_API_VERSION='v3'

youtube=build(YOUTUBE_API_SERVICE_NAME,YOUTUBE_API_VERSION,developerKey=DEVELOPER_KEY)

for filename in file_lst:
    datas = pd.read_csv(f".\DATA\ids\{filename}")

    channelId, playlistName = filename.split('_')
    get_statics(datas,channelId, playlistName)


dict_keys(['publishedAt', 'channelId', 'title', 'description', 'thumbnails', 'channelTitle', 'tags', 'categoryId', 'liveBroadcastContent', 'localized', 'defaultAudioLanguage'])
dict_keys(['publishedAt', 'channelId', 'title', 'description', 'thumbnails', 'channelTitle', 'tags', 'categoryId', 'liveBroadcastContent', 'localized', 'defaultAudioLanguage'])
dict_keys(['publishedAt', 'channelId', 'title', 'description', 'thumbnails', 'channelTitle', 'tags', 'categoryId', 'liveBroadcastContent', 'localized', 'defaultAudioLanguage'])
dict_keys(['publishedAt', 'channelId', 'title', 'description', 'thumbnails', 'channelTitle', 'tags', 'categoryId', 'liveBroadcastContent', 'localized', 'defaultAudioLanguage'])
dict_keys(['publishedAt', 'channelId', 'title', 'description', 'thumbnails', 'channelTitle', 'tags', 'categoryId', 'liveBroadcastContent', 'localized', 'defaultAudioLanguage'])
dict_keys(['publishedAt', 'channelId', 'title', 'description', 'thumbnails', 'channelTitle', 'categoryId', 'liveBro

KeyError: 'tags'

In [1]:
testStr = '가나다_라마바'
st1, st2 = testStr.split('_')
print(st1,st2)

가나다 라마바
